In [5]:
import pickle
import os
import inflection
import math
import datetime
import pandas as pd
import numpy  as np
from flask import Flask, request, Response
import requests

df_store = pd.read_csv('\\Users\\micro\\Desktop\\repos\\3_ciclo_intermediario\\1_ds_em_producao\\data\\store.csv', low_memory = False)
df = pd.read_csv('\\Users\\micro\\Desktop\\repos\\3_ciclo_intermediario\\1_ds_em_producao\\data\\test.csv')

In [6]:
# merge test dataset + store (perfomance das lojas + características das lojas)
df_test = pd.merge(df, df_store, how = 'left', on = 'Store')

# lojas escolhidas para testar as predições
df_test = df_test[df_test['Store'].isin([20, 23, 22])]

# removendo do teste os dias fechados
df_test = df_test[df_test['Open'] != 0]
df_test = df_test[~df_test['Open'].isnull()] # ~: diferente
df_test = df_test.drop('Id', axis = 1)

In [7]:
# converte o dataframe em json para fazer a comunicação entre os sistemas
data = json.dumps(df_test.to_dict(orient = 'records'))

In [8]:
# chamada em API
url = 'https://rossmann-api-5z78.onrender.com/rossmann/predict'
header = {'Content-type': 'application/json'} 
data = data

r = requests.post(url, data = data, headers = header) # post: precisa enviar algum dado para requerir
print('Status Code {}'.format(r.status_code)) # para ver se a requisição é válida ou não 

Status Code 500


In [ ]:
# pega o json da requisição e converte em dataframe
d1 = pd.DataFrame(r.json(), columns = r.json()[0].keys())
d1.head()

In [ ]:
# mensagem mostrada ao usuário
d2 = d1[['store', 'prediction']].groupby('store').sum().reset_index()

for i in range(len(d2)):
    print('A loja {} irá vender R${:,.2f} nas próximas 6 semanas'.format(d2.loc[i, 'store'], d2.loc[i, 'prediction']))